In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

# Designate variables
llm = ChatOpenAI(
    temperature=0.1,
    model_name="gpt-4o"
)

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages= True,
)

loader = TextLoader(r"./files/chapter_3.txt")

splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cache_dir = LocalFileStore("./cache/")

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir    
)

vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriever = vectorstore.as_retriever()

# List of docs generation
map_doc_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim
    ------------------
    {portion}
    """),
    ("human", "{question}"),
])
map_doc_chain = map_doc_prompt | llm

# Merge list of docs
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(map_doc_chain.invoke({
        "portion": doc.page_content,
        "question": question
    }).content for doc in documents)

map_chain = {"documents": retriever, "question": RunnablePassthrough()} | RunnableLambda(map_docs)

# Final document | prompt | llm
final_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    Give the following extracted parts of a long document and a question, create a final answer.
    If you don't know the answer, just say that you don't know. Don't try to make up the answer.
    ---------
    {context}
    """),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])

def load_memory(input):
    return memory.load_memory_variables({})["chat_history"]

chain = {"context": map_chain, "question": RunnablePassthrough()} | RunnablePassthrough.assign(chat_history=load_memory) | final_prompt | llm

def invoke_chain(question):
    result=chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )
    print(result)

invoke_chain("Is Aaronson guilty?")


In [ ]:
invoke_chain("What message did he write in the table?")

In [ ]:
invoke_chain("Who is Julia?")